In [2]:
import sys
import numpy as np
import scipy.io
from pyDOE import lhs
from torch import Tensor, ones, stack, load
from torch.autograd import grad
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [3]:
sys.path.append('/Users/juanesteban') 
import PINNFramework as pf

Was not able to import Horovod. Thus Horovod support is not enabled


In [4]:
class BoundaryConditionDataset(Dataset):

    def __init__(self, nb, lb, ub):
        """
        Constructor of the initial condition dataset
        Args:
          n0 (int)
        """
        super(type(self)).__init__()
        data = scipy.io.loadmat('/Users/juanesteban/PINNS/NeuralSolvers/examples/1D_Schroedinger/NLS.mat')
        t = data['tt'].flatten()[:, None]
        idx_t = np.random.choice(t.shape[0], nb, replace=False)
        tb = t[idx_t, :]
        self.x_lb = np.concatenate((0 * tb + lb[0], tb), 1)  # (lb[0], tb)
        self.x_ub = np.concatenate((0 * tb + ub[0], tb), 1)  # (ub[0], tb)

    def __getitem__(self, idx):
        """
        Returns data for initial state
        """
        return Tensor(self.x_lb).float(), Tensor(self.x_ub).float()

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1


In [12]:
class InitialConditionDataset(Dataset):

    def __init__(self, n0):
        """
        Constructor of the boundary condition dataset
        Args:
          n0 (int)
        """
        super(type(self)).__init__()
        data = scipy.io.loadmat('/Users/juanesteban/PINNS/NeuralSolvers/examples/1D_Schroedinger/NLS.mat')
        x = data['x'].flatten()[:, None]
        t = data['tt'].flatten()[:, None]
        Exact = data['uu']
        Exact_u = np.real(Exact)
        Exact_v = np.imag(Exact)
        idx_x = np.random.choice(x.shape[0], n0, replace=False)
        self.x = x[idx_x, :]
        print(self.x.shape)
        self.u = Exact_u[idx_x, 0:1]
        self.v = Exact_v[idx_x, 0:1]
        self.t = np.zeros(self.x.shape)

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1

    def __getitem__(self, idx):
        x = np.concatenate([self.x, self.t], axis=1)
        print(x)
        y = np.concatenate([self.u, self.v], axis=1)
        print(x.shape)
        return Tensor(x).float(), Tensor(y)



In [13]:
class PDEDataset(Dataset):
    def __init__(self, nf, lb, ub):
        self.xf = lb + (ub - lb) * lhs(2, nf)

    def __getitem__(self, idx):
        """
        Returns data for initial state
        """
        return Tensor(self.xf).float()

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1


if __name__ == "__main__":
    # Domain bounds
    lb = np.array([-5.0, 0.0])
    ub = np.array([5.0, np.pi / 2])
    # initial condition
    ic_dataset = InitialConditionDataset(n0=50)
    initial_condition = pf.InitialCondition(ic_dataset)
    # boundary conditions
    bc_dataset = BoundaryConditionDataset(nb=50, lb=lb, ub=ub)
    periodic_bc_u = pf.PeriodicBC(bc_dataset, 0, "u periodic boundary condition")
    periodic_bc_v = pf.PeriodicBC(bc_dataset, 1, "v periodic boundary condition")
    periodic_bc_u_x = pf.PeriodicBC(bc_dataset, 0, "u_x periodic boundary condition", 1, 0)
    periodic_bc_v_x = pf.PeriodicBC(bc_dataset, 1, "v_x periodic boundary condition", 1, 0)
    # PDE
    pde_dataset = PDEDataset(20000, lb, ub)


    def schroedinger1d(x, u):
        pred = u
        u = pred[:, 0]
        v = pred[:, 1]
        print("x:", x.shape)
        print("u:", v.shape)
        grads = ones(u.shape, device=pred.device) # move to the same device as prediction
        grad_u = grad(u, x, create_graph=True, grad_outputs=grads)[0]
        grad_v = grad(v, x, create_graph=True, grad_outputs=grads)[0]

        # calculate first order derivatives
        u_x = grad_u[:, 0]
        print("u_x", u_x.shape)
        u_t = grad_u[:, 1]

        v_x = grad_v[:, 0]
        v_t = grad_v[:, 1]

        # calculate second order derivatives
        grad_u_x = grad(u_x, x, create_graph=True, grad_outputs=grads)[0]
        grad_v_x = grad(v_x, x, create_graph=True, grad_outputs=grads)[0]

        u_xx = grad_u_x[:, 0]
        print("u_xx", u_xx.shape)
        v_xx = grad_v_x[:, 0]
        f_u = u_t + u_x#0.5 * v_xx + (u ** 2 + v ** 2) * v
        print("f_u.shape", f_u.shape)
        f_v = v_t - 0.5 * u_xx - (u ** 2 + v ** 2) * u
        print(f_u)
        return stack([f_u, f_v], 1)  # concatenate real part and imaginary part


    pde_loss = pf.PDELoss(pde_dataset, schroedinger1d)
    model = pf.models.MLP(input_size=2, output_size=2, hidden_size=100, num_hidden=4, lb=lb, ub=ub)
    pinn = pf.PINN(model, 2, 2, pde_loss, initial_condition, [periodic_bc_u,
                                                              periodic_bc_v,
                                                              periodic_bc_u_x,
                                                              periodic_bc_v_x], use_gpu=False)
    pinn.fit(2, 'Adam', 1e-3)

(50, 1)
[[-3.3203125  0.       ]
 [ 4.53125    0.       ]
 [ 3.984375   0.       ]
 [-2.109375   0.       ]
 [ 1.328125   0.       ]
 [ 2.421875   0.       ]
 [-2.3046875  0.       ]
 [-0.78125    0.       ]
 [ 3.7109375  0.       ]
 [ 1.8359375  0.       ]
 [-1.015625   0.       ]
 [ 3.4375     0.       ]
 [ 3.828125   0.       ]
 [-2.890625   0.       ]
 [ 1.640625   0.       ]
 [ 4.375      0.       ]
 [-0.2734375  0.       ]
 [-3.0078125  0.       ]
 [ 0.6640625  0.       ]
 [ 0.1953125  0.       ]
 [ 4.8828125  0.       ]
 [ 0.703125   0.       ]
 [ 0.         0.       ]
 [-4.3359375  0.       ]
 [-4.6875     0.       ]
 [ 0.1171875  0.       ]
 [-2.578125   0.       ]
 [-3.9453125  0.       ]
 [-0.859375   0.       ]
 [ 1.484375   0.       ]
 [-1.09375    0.       ]
 [-4.140625   0.       ]
 [ 2.8125     0.       ]
 [ 3.8671875  0.       ]
 [ 2.8515625  0.       ]
 [ 0.3515625  0.       ]
 [ 1.7578125  0.       ]
 [-1.2890625  0.       ]
 [-2.0703125  0.       ]
 [-4.296875   0. 

KeyboardInterrupt: 

In [ ]:
 # Plotting
data = scipy.io.loadmat('/Users/juanesteban/PINNS/NeuralSolvers/examples/1D_Schroedinger/NLS.mat')
t = data['tt'].flatten()[:, None]
x = data['x'].flatten()[:, None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
Exact_h = np.sqrt(Exact_u ** 2 + Exact_v ** 2)
X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact_u.T.flatten()[:, None]
v_star = Exact_v.T.flatten()[:, None]
h_star = Exact_h.T.flatten()[:, None]

pred = model(Tensor(X_star).cpu())
pred_u = pred[:, 0].detach().cpu().numpy()
pred_v = pred[:, 1].detach().cpu().numpy()
H_pred = np.sqrt(pred_u ** 2 + pred_v**2)
H_pred = H_pred.reshape(X.shape)
plt.imshow(pred_u.reshape(X.shape).T, interpolation='nearest', cmap='YlGnBu',
              extent= [lb[1], ub[1], lb[0], ub[0]],
              origin='lower', aspect='auto')
plt.xlabel('Time $t$')
plt.ylabel('Position $x$')
plt.colorbar()
plt.show()

In [8]:
import torch
from torch import Tensor as Tensor
from torch.nn import Module as Module
from torch.nn import MSELoss, L1Loss
from PINNFramework.LossTerm import LossTerm


class PDE_Loss(LossTerm):
    def __init__(self, dataset, pde, norm='L2', weight=1.):
        """
        Constructor of the PDE Loss
        Args:
            dataset (torch.utils.Dataset): dataset that provides the residual points
            pde (function): function that represents residual of the PDE
            norm: Norm used for calculation PDE loss
            weight: Weighting for the loss term
        """
        super(PDELoss, self).__init__(dataset, norm, weight)
        self.dataset = dataset
        self.pde = pde
    def i_star(f_i):
        C_b=torch.fft(f_i,1)
        f_star=[complex(*C_b[j])/(2*np.pi*j)*
                np.exp(2*np.pi*np.exp(complex(0,1)*j*2)) for j in range(1,len(C_b))]
        return [complex(*C_b[0]),*f_star]
    def __call__(self, x: Tensor, model: Module, **kwargs):
        """
        Call function of the PDE loss. Calculates the norm of the PDE residual
        x: residual points
        model: model that predicts the solution of the PDE
        """
        x.requires_grad = True  # setting requires grad to true in order to calculate
        u = model.forward(x)
        pde_residual = self.pde(x, u, **kwargs)
        #pde_residual = i_star(pde_residual)#torch.fft(pde_residual,1)
        zeros = torch.zeros(pde_residual.shape, device=pde_residual.device)
        return self.norm(pde_residual, zeros)

In [9]:
class PDEDataset(Dataset):
    def __init__(self, nf, lb, ub):
        self.xf = lb + (ub - lb) * lhs(2, nf)

    def __getitem__(self, idx):
        """
        Returns data for initial state
        """
        return Tensor(self.xf).float()

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1


if __name__ == "__main__":
    # Domain bounds
    lb = np.array([-5.0, 0.0])
    ub = np.array([5.0, np.pi / 2])
    # initial condition
    ic_dataset = InitialConditionDataset(n0=50)
    initial_condition = pf.InitialCondition(ic_dataset)
    # boundary conditions
    bc_dataset = BoundaryConditionDataset(nb=50, lb=lb, ub=ub)
    periodic_bc_u = pf.PeriodicBC(bc_dataset, 0, "u periodic boundary condition")
    periodic_bc_v = pf.PeriodicBC(bc_dataset, 1, "v periodic boundary condition")
    periodic_bc_u_x = pf.PeriodicBC(bc_dataset, 0, "u_x periodic boundary condition", 1, 0)
    periodic_bc_v_x = pf.PeriodicBC(bc_dataset, 1, "v_x periodic boundary condition", 1, 0)
    # PDE
    pde_dataset = PDEDataset(20000, lb, ub)


    def schroedinger1d(x, u):
        pred = u
        u = pred[:, 0]
        v = pred[:, 1]
        print("x:", x.shape)
        print("u:", v.shape)
        grads = ones(u.shape, device=pred.device) # move to the same device as prediction
        grad_u = grad(u, x, create_graph=True, grad_outputs=grads)[0]
        grad_v = grad(v, x, create_graph=True, grad_outputs=grads)[0]

        # calculate first order derivatives
        u_x = grad_u[:, 0]
        print("u_x", u_x.shape)
        u_t = grad_u[:, 1]

        v_x = grad_v[:, 0]
        v_t = grad_v[:, 1]

        # calculate second order derivatives
        grad_u_x = grad(u_x, x, create_graph=True, grad_outputs=grads)[0]
        grad_v_x = grad(v_x, x, create_graph=True, grad_outputs=grads)[0]

        u_xx = grad_u_x[:, 0]
        print("u_xx", u_xx.shape)
        v_xx = grad_v_x[:, 0]
        f_u = u_t + u_x#0.5 * v_xx + (u ** 2 + v ** 2) * v
        print("f_u.shape", f_u.shape)
        f_v = 0*v_t# - 0.5 * u_xx - (u ** 2 + v ** 2) * u
        return stack([f_u, f_v], 1)  # concatenate real part and imaginary part


    pde_loss = pf.PDELoss(pde_dataset, schroedinger1d)
    model = pf.models.MLP(input_size=2, output_size=2, hidden_size=100, num_hidden=4, lb=lb, ub=ub)
    pinn = pf.PINN(model, 2, 2, pde_loss, initial_condition, [periodic_bc_u,
                                                              periodic_bc_v,
                                                              periodic_bc_u_x,
                                                              periodic_bc_v_x], use_gpu=False)
    pinn.fit(2, 'Adam', 1e-3)

[[-1.484375   0.       ]
 [-0.9765625  0.       ]
 [-3.4375     0.       ]
 [ 3.046875   0.       ]
 [-1.015625   0.       ]
 [-1.640625   0.       ]
 [ 3.359375   0.       ]
 [-3.75       0.       ]
 [ 4.2578125  0.       ]
 [-1.9921875  0.       ]
 [-4.0234375  0.       ]
 [-2.5390625  0.       ]
 [-1.171875   0.       ]
 [-0.15625    0.       ]
 [-0.1953125  0.       ]
 [ 1.8359375  0.       ]
 [-0.859375   0.       ]
 [ 3.4375     0.       ]
 [ 4.6484375  0.       ]
 [ 3.59375    0.       ]
 [ 0.4296875  0.       ]
 [ 1.953125   0.       ]
 [ 4.453125   0.       ]
 [-4.4921875  0.       ]
 [-2.109375   0.       ]
 [ 2.0703125  0.       ]
 [-2.03125    0.       ]
 [-3.7109375  0.       ]
 [-4.0625     0.       ]
 [ 3.671875   0.       ]
 [-0.546875   0.       ]
 [ 3.28125    0.       ]
 [ 3.7109375  0.       ]
 [-4.5703125  0.       ]
 [ 0.9765625  0.       ]
 [-0.2734375  0.       ]
 [-4.1015625  0.       ]
 [-3.203125   0.       ]
 [ 1.25       0.       ]
 [ 1.6796875  0.       ]


KeyboardInterrupt: 

In [10]:
inp =  torch.rand([4, 2])
C_b=torch.fft(inp,1)

tensor([[0.4034, 0.1033],
        [0.5311, 0.8406],
        [0.0194, 0.5302],
        [0.2771, 0.7710]])

In [299]:
def FT(f_i,x, shift):
    C_b=torch.fft(f_i,1)
    N_2 = int(len(f_i)/2)
    f_star = np.exp(complex(0,2*np.pi*shift*torch.sum(x)))*np.sum([complex(*np.array(C_b[b]))*
                                        np.exp(complex(0,2*np.pi*b*(torch.sum(x))))
                     for b in range(-N_2,N_2)])
    return [f_star.real, f_star.imag]

In [300]:
x_t=torch.tensor([[i,2*i] for i in range(4)])

In [301]:
f_xt= torch.Tensor([[i,i] for i in range(4)])

In [392]:
def FT(f_i,x, shift):
    C_b=torch.fft(f_i,1)
    N_2 = int(len(f_i)/2)
    zer = torch.Tensor([0])
    im_shift = torch.Tensor([2*np.pi*shift*torch.sum(x)])
    F_y= torch.tensor([torch.complex(C_b[b][0],C_b[b][1])*
                       torch.exp(torch.complex(zer,torch.Tensor([2*np.pi*b*(torch.sum(x))])))
                       for b in range(-N_2,N_2)])
    f_star = (torch.exp(torch.complex(zer,im_shift))*torch.sum(F_y))
    print(f_star)
    return torch.tensor([torch.real(f_star), torch.imag(f_star)],grad_fn='AddmmBackward')

In [393]:
r=FT(f_xt,x_t,1)
torch.stack([r for i in range(3)],0)

tensor([-2.3842e-06-1.9111e-07j])


TypeError: tensor() got an unexpected keyword argument 'grad_fn'

In [362]:
zer = torch.Tensor([0])
torch.complex(zer,zer)

tensor([0.+0.j])

In [401]:
inv_fthsqrt_pi = 1 / np.pi ** (1. / 4.)
inv_sqrt_two = 1. / np.sqrt(2.)

def phi0(x,f):
    root_f = f ** (1./4.)
    return root_f*inv_fthsqrt_pi * np.exp(- x * f * x / 2.0)

def phi1(x,f):
    return  phi0(x,f) * inv_sqrt_two * (np.sqrt(f) * 2.0 * x)

def Phi0(x, y, f):
    return phi0(x,f) * phi0(y,f)

def Phi1(x, y, f):
    return phi1(x,f) * phi1(y,f)


def Psi(x, y, t,f):
    return inv_sqrt_two * (np.exp(-1j*f*t) * Phi0(x, y,f) + np.exp(-1j*3*f*t)*Phi1(x, y,f))


def write_solution(x, y, t, f, prefix, filebase="step-"):
    sol = Psi(x, y, t, f).T.reshape(200, 200)
    dt = 0.001
    u = sol.real
    v = sol.imag
    u = u.reshape(-1)
    v = v.reshape(-1)
    state = int(t / dt)
    with h5.File(os.path.join(prefix,"{}{}.h5".format(filebase,state)), "w") as f:
        f.create_dataset('real', data=u)
        f.create_dataset('imag', data=v)

def main():

    parser = argparse.ArgumentParser(description='Calculate Analytical Solution for Quantum Oscillator in 2D')
    # parser.add_argument('integers', metavar='N', type=int, nargs='+',
    #                     help='an integer for the accumulator')
    parser.add_argument('--xdim', action='store', type=int,
                        default=200,
                        help='number of samples in x, arbitrary units')
    parser.add_argument('--ydim', action='store', type=int,
                        default=200,
                        help='number of samples in y, arbitrary units')
    parser.add_argument('--dt', action='store', type=float,
                        default=.001,
                        help='time delta in arbitrary units')
    parser.add_argument('--nsteps', action='store', type=int,
                        default=2000,
                        help='number of time steps')
    parser.add_argument('--prefix', action='store',
                        default="analytical_results/",type=str,
                        help='prefix to store analytical_results into')
    parser.add_argument('--filebase', action='store',
                        default="step-",type=str,
                        help='file base name to use, e.g. step- will produce step-1.h5 etc.')
    parser.add_argument('--f', action='store',
                        default=1.,type=float,
                        help='Frequency of harmonic oscillator')
    args = parser.parse_args()
    print(args.prefix)
    x = np.linspace(-3,3, args.xdim)
    y = np.linspace(-3,3, args.ydim)

    dt = args.dt
    X, Y = np.meshgrid(x, y)

    X = X.reshape(-1)
    Y = Y.reshape(-1)

    for t in range(0,args.nsteps):
        if t % 200 == 0:
            print("t", t)
        write_solution(X, Y, t*dt,args.f, args.prefix,args.filebase)

if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] [--xdim XDIM] [--ydim YDIM] [--dt DT]
                             [--nsteps NSTEPS] [--prefix PREFIX]
                             [--filebase FILEBASE] [--f F]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/juanesteban/Library/Jupyter/runtime/kernel-c05fac36-cadd-4082-8eef-eb2075e75bb3.json


SystemExit: 2

In [432]:
x = np.linspace(-3,3, 200)
y = np.linspace(-3,3, 200)

dt = 0.001
X, Y = np.meshgrid(x, y)

X = X.reshape(-1)
Y = Y.reshape(-1)
for t in range(0,2000):
    if t % 200 == 0:
        print("t", t)
sol = Psi(X, Y, t, 1).T.reshape(200, 200)
dt = 0.001
u = sol.real
v = sol.imag
u = u.reshape(-1)
v = v.reshape(-1)
state = int(t / dt)
with h5.File(os.path.join("{}{}.h5".format('step-',state)), "w") as f:
    f.create_dataset('real', data=u)
    f.create_dataset('imag', data=v)

t 0
t 200
t 400
t 600
t 800
t 1000
t 1200
t 1400
t 1600
t 1800
